# LoRA-Diffusion Results Analysis

This notebook provides analysis tools for LoRA-Diffusion experiments.

**Contents:**
1. Load and visualize training history
2. Compare methods across tasks
3. Analyze parameter efficiency
4. Visualize step-adaptive ranks
5. Performance vs. efficiency tradeoffs

In [ ]:
# Imports
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import numpy as np

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

print("Imports successful!")

## 1. Load Training History

In [ ]:
def load_training_history(output_dir):
    """Load training history from output directory."""
    history_file = Path(output_dir) / "training_history.json"
    
    if not history_file.exists():
        print(f"Warning: {history_file} not found")
        return None
    
    with open(history_file, 'r') as f:
        history = json.load(f)
    
    return pd.DataFrame(history)

# Example: Load a specific experiment
output_dir = "./outputs/sst2_lora_diffusion"  # Change this to your output directory

df = load_training_history(output_dir)

if df is not None:
    print(f"Loaded {len(df)} training steps")
    print("\nColumns:", df.columns.tolist())
    print("\nFirst few rows:")
    display(df.head())
else:
    print("No training history found. Run training first.")

## 2. Visualize Training Curves

In [ ]:
def plot_training_curves(df, metrics=['loss', 'accuracy']):
    """Plot training curves."""
    if df is None:
        print("No data to plot")
        return
    
    fig, axes = plt.subplots(1, len(metrics), figsize=(6*len(metrics), 5))
    
    if len(metrics) == 1:
        axes = [axes]
    
    for ax, metric in zip(axes, metrics):
        # Extract metric values
        if 'metrics' in df.columns:
            values = df['metrics'].apply(lambda x: x.get(metric, np.nan))
        else:
            values = df.get(metric, [])
        
        ax.plot(df['step'], values, linewidth=2)
        ax.set_xlabel('Training Step')
        ax.set_ylabel(metric.replace('_', ' ').title())
        ax.set_title(f'Training {metric.replace("_", " ").title()}')
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Plot training curves
if df is not None:
    plot_training_curves(df, metrics=['loss', 'accuracy'])

## 3. Compare Multiple Methods

In [ ]:
def load_multiple_experiments(base_dir, experiment_names):
    """Load multiple experiment results."""
    results = {}
    
    for name in experiment_names:
        exp_dir = Path(base_dir) / name
        df = load_training_history(exp_dir)
        if df is not None:
            results[name] = df
    
    return results

# Example: Compare different methods on SST-2
experiment_names = [
    "sst2_lora_diffusion",
    "sst2_weight_lora",
    "sst2_full_ft",
    "sst2_adapters",
]

base_dir = "./outputs"
experiments = load_multiple_experiments(base_dir, experiment_names)

print(f"Loaded {len(experiments)} experiments")
for name in experiments:
    print(f"  - {name}: {len(experiments[name])} steps")

In [ ]:
def plot_method_comparison(experiments, metric='loss'):
    """Compare multiple methods."""
    plt.figure(figsize=(12, 6))
    
    for name, df in experiments.items():
        if 'metrics' in df.columns:
            values = df['metrics'].apply(lambda x: x.get(metric, np.nan))
        else:
            values = df.get(metric, [])
        
        # Clean name for legend
        method_name = name.split('_', 1)[1].replace('_', ' ').title()
        plt.plot(df['step'], values, label=method_name, linewidth=2)
    
    plt.xlabel('Training Step')
    plt.ylabel(metric.replace('_', ' ').title())
    plt.title(f'Method Comparison: {metric.replace("_", " ").title()}')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

# Plot comparison
if experiments:
    plot_method_comparison(experiments, metric='loss')
    plot_method_comparison(experiments, metric='accuracy')

## 4. Parameter Efficiency Analysis

In [ ]:
def analyze_parameter_efficiency(checkpoint_dirs):
    """Analyze parameter efficiency of different methods."""
    results = []
    
    for name, checkpoint_dir in checkpoint_dirs.items():
        config_file = Path(checkpoint_dir) / "config.json"
        
        if not config_file.exists():
            continue
        
        with open(config_file, 'r') as f:
            config = json.load(f)
        
        # Calculate parameter counts (simplified)
        # In practice, you'd load the actual model
        method = config.get('method', {}).get('name', 'unknown')
        
        # Approximate parameter counts
        param_counts = {
            'lora_diffusion': 0.7,
            'weight_lora': 0.9,
            'adapters': 2.1,
            'prefix_tuning': 1.0,
            'bitfit': 0.1,
            'full_ft': 100.0,
        }
        
        results.append({
            'Method': name.split('_', 1)[1].replace('_', ' ').title(),
            'Trainable %': param_counts.get(method, 1.0),
        })
    
    return pd.DataFrame(results)

# Example parameter efficiency data
param_data = pd.DataFrame([
    {'Method': 'LoRA-Diffusion', 'Trainable %': 0.7, 'Performance': 98.1},
    {'Method': 'Weight LoRA', 'Trainable %': 0.9, 'Performance': 94.1},
    {'Method': 'Adapters', 'Trainable %': 2.1, 'Performance': 96.1},
    {'Method': 'Prefix Tuning', 'Trainable %': 1.0, 'Performance': 92.1},
    {'Method': 'BitFit', 'Trainable %': 0.1, 'Performance': 86.5},
    {'Method': 'Full FT', 'Trainable %': 100.0, 'Performance': 100.0},
])

display(param_data)

In [ ]:
def plot_efficiency_frontier(df):
    """Plot performance vs. parameter efficiency."""
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Scatter plot
    scatter = ax.scatter(
        df['Trainable %'],
        df['Performance'],
        s=200,
        alpha=0.6,
        c=range(len(df)),
        cmap='viridis'
    )
    
    # Add labels
    for _, row in df.iterrows():
        ax.annotate(
            row['Method'],
            (row['Trainable %'], row['Performance']),
            xytext=(5, 5),
            textcoords='offset points',
            fontsize=10,
            bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.7)
        )
    
    ax.set_xlabel('Trainable Parameters (%)', fontsize=12)
    ax.set_ylabel('Performance (% of Full FT)', fontsize=12)
    ax.set_title('Parameter Efficiency Frontier', fontsize=14, fontweight='bold')
    ax.set_xscale('log')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

plot_efficiency_frontier(param_data)

## 5. Step-Adaptive Rank Visualization

In [ ]:
def visualize_rank_schedule(num_steps=100):
    """Visualize step-adaptive rank allocation."""
    # Define rank schedule
    steps = np.arange(num_steps)
    ranks = np.zeros(num_steps)
    scalings = np.zeros(num_steps)
    
    # Apply schedule
    for t in range(num_steps):
        if t > 2 * num_steps // 3:  # Early
            ranks[t] = 64
            scalings[t] = 1.0
        elif t > num_steps // 3:  # Mid
            ranks[t] = 32
            scalings[t] = 0.5
        else:  # Late
            ranks[t] = 8
            scalings[t] = 0.25
    
    # Plot
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)
    
    # Rank
    ax1.plot(steps, ranks, linewidth=3, color='steelblue')
    ax1.set_ylabel('Rank (r)', fontsize=12)
    ax1.set_title('Step-Adaptive Rank Allocation', fontsize=14, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    ax1.set_ylim([0, 70])
    
    # Add phase labels
    ax1.axvspan(67, 100, alpha=0.2, color='red', label='Early (r=64)')
    ax1.axvspan(34, 67, alpha=0.2, color='orange', label='Mid (r=32)')
    ax1.axvspan(0, 34, alpha=0.2, color='green', label='Late (r=8)')
    ax1.legend(loc='upper right')
    
    # Scaling
    ax2.plot(steps, scalings, linewidth=3, color='coral')
    ax2.set_xlabel('Diffusion Step (t)', fontsize=12)
    ax2.set_ylabel('Scaling (σ)', fontsize=12)
    ax2.set_title('Step-Adaptive Scaling', fontsize=14, fontweight='bold')
    ax2.grid(True, alpha=0.3)
    ax2.set_ylim([0, 1.1])
    
    plt.tight_layout()
    plt.show()

visualize_rank_schedule()

## 6. Load and Compare Final Results

In [ ]:
def load_evaluation_results(result_files):
    """Load evaluation results from multiple experiments."""
    results = []
    
    for name, file_path in result_files.items():
        file_path = Path(file_path)
        if not file_path.exists():
            continue
        
        with open(file_path, 'r') as f:
            data = json.load(f)
        
        metrics = data.get('metrics', {})
        metrics['Method'] = name
        results.append(metrics)
    
    return pd.DataFrame(results)

# Example: Load results (update paths as needed)
result_files = {
    'LoRA-Diffusion': './outputs/sst2_lora_diffusion/eval_results.json',
    'Weight LoRA': './outputs/sst2_weight_lora/eval_results.json',
    'Full FT': './outputs/sst2_full_ft/eval_results.json',
}

# results_df = load_evaluation_results(result_files)
# display(results_df)

print("Note: Update result_files paths with your actual output directories")

## 7. Multi-Task Comparison

In [ ]:
def create_comparison_heatmap(data):
    """Create heatmap comparing methods across tasks."""
    plt.figure(figsize=(10, 6))
    
    sns.heatmap(
        data,
        annot=True,
        fmt='.1f',
        cmap='RdYlGn',
        center=80,
        vmin=70,
        vmax=95,
        cbar_kws={'label': 'Performance Score'}
    )
    
    plt.title('Method Performance Across Tasks', fontsize=14, fontweight='bold')
    plt.xlabel('Task', fontsize=12)
    plt.ylabel('Method', fontsize=12)
    plt.tight_layout()
    plt.show()

# Example data (from paper Table 3)
comparison_data = pd.DataFrame({
    'SST-2': [94.2, 92.1, 93.4, 91.8, 94.5],
    'SQuAD': [87.9, 84.3, 86.9, 83.1, 88.7],
    'XSum': [37.4, 35.2, 36.9, 34.3, 37.8],
}, index=['LoRA-Diffusion', 'Weight LoRA', 'Adapters', 'Prefix', 'Full FT'])

create_comparison_heatmap(comparison_data)

## 8. Training Time Analysis

In [ ]:
def plot_training_time_comparison(data):
    """Compare training times."""
    fig, ax = plt.subplots(figsize=(10, 6))
    
    colors = plt.cm.viridis(np.linspace(0, 0.9, len(data)))
    bars = ax.barh(data['Method'], data['Time (hours)'], color=colors)
    
    ax.set_xlabel('Training Time (hours)', fontsize=12)
    ax.set_title('Training Time Comparison (SST-2, 5000 steps)', fontsize=14, fontweight='bold')
    ax.grid(axis='x', alpha=0.3)
    
    # Add value labels
    for i, (bar, time) in enumerate(zip(bars, data['Time (hours)'])):
        ax.text(time + 0.02, bar.get_y() + bar.get_height()/2, 
                f'{time:.2f}h', va='center', fontsize=10)
    
    plt.tight_layout()
    plt.show()

# Example data
time_data = pd.DataFrame([
    {'Method': 'LoRA-Diffusion', 'Time (hours)': 0.45},
    {'Method': 'Weight LoRA', 'Time (hours)': 0.54},
    {'Method': 'Adapters', 'Time (hours)': 0.72},
    {'Method': 'Full FT', 'Time (hours)': 0.91},
])

plot_training_time_comparison(time_data)

## 9. Summary Statistics

In [ ]:
def print_summary_statistics(experiments):
    """Print summary statistics for experiments."""
    print("=" * 80)
    print("EXPERIMENT SUMMARY")
    print("=" * 80)
    
    for name, df in experiments.items():
        print(f"\n{name}:")
        print("-" * 40)
        
        # Get final metrics
        if 'metrics' in df.columns and len(df) > 0:
            final_metrics = df.iloc[-1]['metrics']
            print(f"  Final Loss: {final_metrics.get('loss', 'N/A'):.4f}")
            print(f"  Final Accuracy: {final_metrics.get('accuracy', 'N/A'):.4f}")
        
        # Training time
        if 'time_per_step' in df.columns:
            total_time = df['time_per_step'].sum() / 3600  # Convert to hours
            print(f"  Total Training Time: {total_time:.2f} hours")
    
    print("\n" + "=" * 80)

if experiments:
    print_summary_statistics(experiments)

## 10. Export Results for Paper

In [ ]:
def export_latex_table(df, caption="Results comparison", label="tab:results"):
    """Export results as LaTeX table."""
    latex = df.to_latex(
        index=True,
        float_format="%.2f",
        caption=caption,
        label=label,
        escape=False,
    )
    
    print("LaTeX Table:")
    print("=" * 80)
    print(latex)
    print("=" * 80)
    
    # Save to file
    with open('results_table.tex', 'w') as f:
        f.write(latex)
    print("\nSaved to results_table.tex")

# Example
# export_latex_table(comparison_data, caption="Performance across tasks", label="tab:performance")

## 11. Generate Paper Figures

Generate the required figures for the paper as specified in `doc/figures/README.md`:
1. Rank ablation study
2. Effective rank across diffusion steps
3. Data efficiency comparison
4. Trajectory visualization (t-SNE)

In [ ]:
# Setup output directory for figures
from pathlib import Path
import os

figures_dir = Path("../doc/figures")
figures_dir.mkdir(parents=True, exist_ok=True)
print(f"Figures will be saved to: {figures_dir.absolute()}")

### Figure 1: Rank Ablation Study

In [ ]:
def plot_rank_ablation():
    """Generate rank ablation figure: Rank vs. performance (left) and vs. trainable parameters (right)."""
    
    # Example data - replace with actual experimental results
    # Rank configurations: fixed ranks vs. step-adaptive (8/32/64)
    ranks = [4, 8, 16, 32, 64, 128, '8/32/64 (adaptive)']
    performance = [88.5, 91.2, 93.8, 95.1, 95.8, 96.2, 96.5]  # Performance scores
    trainable_params = [0.2, 0.4, 0.8, 1.6, 3.2, 6.4, 1.1]  # Trainable params in millions
    
    # Convert adaptive rank to numeric for plotting
    rank_numeric = [4, 8, 16, 32, 64, 128, 35]  # Approximate for adaptive
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    
    # Left plot: Rank vs. Performance
    colors = ['steelblue'] * 6 + ['red']
    markers = ['o'] * 6 + ['*']
    sizes = [100] * 6 + [200]
    
    for i, (r, r_num, perf, c, m, s) in enumerate(zip(ranks, rank_numeric, performance, colors, markers, sizes)):
        if isinstance(r, str):
            ax1.scatter(r_num, perf, c=c, marker=m, s=s, label=r, zorder=3, edgecolors='black', linewidths=2)
        else:
            ax1.scatter(r_num, perf, c=c, marker=m, s=s, zorder=2, edgecolors='black', linewidths=1)
    
    ax1.plot(rank_numeric[:6], performance[:6], '--', alpha=0.3, color='gray', zorder=1)
    ax1.set_xlabel('Rank (r)', fontsize=14, fontweight='bold')
    ax1.set_ylabel('Performance (%)', fontsize=14, fontweight='bold')
    ax1.set_title('Rank vs. Performance', fontsize=16, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    ax1.legend(loc='lower right', fontsize=11)
    ax1.set_xscale('log', base=2)
    ax1.set_xticks([4, 8, 16, 32, 64, 128])
    ax1.set_xticklabels(['4', '8', '16', '32', '64', '128'])
    
    # Right plot: Rank vs. Trainable Parameters
    for i, (r, r_num, params, c, m, s) in enumerate(zip(ranks, rank_numeric, trainable_params, colors, markers, sizes)):
        if isinstance(r, str):
            ax2.scatter(r_num, params, c=c, marker=m, s=s, label=r, zorder=3, edgecolors='black', linewidths=2)
        else:
            ax2.scatter(r_num, params, c=c, marker=m, s=s, zorder=2, edgecolors='black', linewidths=1)
    
    ax2.plot(rank_numeric[:6], trainable_params[:6], '--', alpha=0.3, color='gray', zorder=1)
    ax2.set_xlabel('Rank (r)', fontsize=14, fontweight='bold')
    ax2.set_ylabel('Trainable Parameters (M)', fontsize=14, fontweight='bold')
    ax2.set_title('Rank vs. Trainable Parameters', fontsize=16, fontweight='bold')
    ax2.grid(True, alpha=0.3)
    ax2.legend(loc='upper left', fontsize=11)
    ax2.set_xscale('log', base=2)
    ax2.set_xticks([4, 8, 16, 32, 64, 128])
    ax2.set_xticklabels(['4', '8', '16', '32', '64', '128'])
    
    plt.tight_layout()
    
    # Save as PNG and PDF
    output_path_png = figures_dir / "rank_ablation.png"
    output_path_pdf = figures_dir / "rank_ablation.pdf"
    plt.savefig(output_path_png, dpi=300, bbox_inches='tight')
    plt.savefig(output_path_pdf, bbox_inches='tight')
    print(f"Saved: {output_path_png}")
    print(f"Saved: {output_path_pdf}")
    plt.show()

plot_rank_ablation()

### Figure 2: Effective Rank Across Diffusion Steps

In [ ]:
def plot_effective_rank():
    """Generate effective rank figure: Effective rank of LoRA modules across diffusion steps."""
    
    # Diffusion steps (t=0 is final step, t=100 is initial noise)
    num_steps = 100
    steps = np.arange(num_steps)
    
    # Effective rank decreases as we go from early (high noise) to late (low noise) steps
    # Early steps (high t) have higher effective rank, late steps (low t) have lower effective rank
    effective_rank = np.zeros(num_steps)
    
    # Simulate effective rank pattern: higher at early steps, lower at late steps
    for t in range(num_steps):
        if t > 2 * num_steps // 3:  # Early steps (t > 66)
            # High effective rank, close to full rank
            effective_rank[t] = 55 + 5 * np.sin(t * 0.1) + np.random.normal(0, 2)
        elif t > num_steps // 3:  # Mid steps (33 < t <= 66)
            # Medium effective rank
            effective_rank[t] = 28 + 3 * np.sin(t * 0.15) + np.random.normal(0, 1.5)
        else:  # Late steps (t <= 33)
            # Lower effective rank
            effective_rank[t] = 6 + 2 * np.sin(t * 0.2) + np.random.normal(0, 1)
    
    # Clip to reasonable bounds
    effective_rank = np.clip(effective_rank, 0, 64)
    
    fig, ax = plt.subplots(figsize=(12, 7))
    
    # Plot effective rank
    ax.plot(steps, effective_rank, linewidth=2.5, color='steelblue', alpha=0.8)
    
    # Add shaded regions for different phases
    ax.axvspan(67, 100, alpha=0.15, color='red', label='Early Steps (High Effective Rank)')
    ax.axvspan(34, 67, alpha=0.15, color='orange', label='Mid Steps (Medium Effective Rank)')
    ax.axvspan(0, 34, alpha=0.15, color='green', label='Late Steps (Low Effective Rank)')
    
    # Add horizontal lines for rank thresholds
    ax.axhline(y=64, color='red', linestyle='--', alpha=0.5, linewidth=1)
    ax.axhline(y=32, color='orange', linestyle='--', alpha=0.5, linewidth=1)
    ax.axhline(y=8, color='green', linestyle='--', alpha=0.5, linewidth=1)
    
    ax.set_xlabel('Diffusion Step (t)', fontsize=14, fontweight='bold')
    ax.set_ylabel('Effective Rank', fontsize=14, fontweight='bold')
    ax.set_title('Effective Rank of LoRA Modules Across Diffusion Steps', fontsize=16, fontweight='bold')
    ax.grid(True, alpha=0.3)
    ax.legend(loc='upper right', fontsize=11)
    ax.set_xlim([0, 100])
    ax.set_ylim([0, 65])
    
    # Invert x-axis to show progression from noise (right) to clean (left)
    # This is more intuitive: early steps (high noise) on right, late steps (clean) on left
    ax.invert_xaxis()
    
    plt.tight_layout()
    
    # Save as PNG and PDF
    output_path_png = figures_dir / "effective_rank.png"
    output_path_pdf = figures_dir / "effective_rank.pdf"
    plt.savefig(output_path_png, dpi=300, bbox_inches='tight')
    plt.savefig(output_path_pdf, bbox_inches='tight')
    print(f"Saved: {output_path_png}")
    print(f"Saved: {output_path_pdf}")
    plt.show()

plot_effective_rank()

### Figure 3: Data Efficiency Comparison

In [ ]:
def plot_data_efficiency():
    """Generate data efficiency figure: Performance vs. training data size."""
    
    # Training data sizes (as percentage of full dataset)
    data_sizes = np.array([10, 20, 30, 50, 70, 100])  # Percentage
    
    # Performance for LoRA-Diffusion (better data efficiency)
    lora_diffusion_perf = np.array([75.2, 82.5, 87.3, 91.8, 94.2, 96.5])
    
    # Performance for Weight LoRA (lower data efficiency)
    weight_lora_perf = np.array([65.1, 72.3, 78.9, 85.2, 89.1, 94.1])
    
    fig, ax = plt.subplots(figsize=(10, 7))
    
    # Plot both methods
    ax.plot(data_sizes, lora_diffusion_perf, 'o-', linewidth=2.5, markersize=10, 
            label='LoRA-Diffusion', color='steelblue', zorder=3)
    ax.plot(data_sizes, weight_lora_perf, 's--', linewidth=2.5, markersize=10, 
            label='Weight LoRA', color='coral', zorder=2, alpha=0.8)
    
    # Fill area between curves to highlight advantage
    ax.fill_between(data_sizes, weight_lora_perf, lora_diffusion_perf, 
                     alpha=0.2, color='steelblue', label='LoRA-Diffusion Advantage')
    
    ax.set_xlabel('Training Data Size (% of Full Dataset)', fontsize=14, fontweight='bold')
    ax.set_ylabel('Performance (%)', fontsize=14, fontweight='bold')
    ax.set_title('Data Efficiency: Performance vs. Training Data Size', fontsize=16, fontweight='bold')
    ax.grid(True, alpha=0.3)
    ax.legend(loc='lower right', fontsize=12)
    ax.set_xlim([5, 105])
    ax.set_ylim([60, 100])
    
    # Add annotations for key points
    ax.annotate('Better data efficiency', 
                xy=(30, 87), xytext=(25, 95),
                arrowprops=dict(arrowstyle='->', color='steelblue', lw=2),
                fontsize=11, color='steelblue', fontweight='bold')
    
    plt.tight_layout()
    
    # Save as PNG and PDF
    output_path_png = figures_dir / "data_efficiency.png"
    output_path_pdf = figures_dir / "data_efficiency.pdf"
    plt.savefig(output_path_png, dpi=300, bbox_inches='tight')
    plt.savefig(output_path_pdf, bbox_inches='tight')
    print(f"Saved: {output_path_png}")
    print(f"Saved: {output_path_pdf}")
    plt.show()

plot_data_efficiency()

### Figure 4: Trajectory Visualization (t-SNE)

In [ ]:
def plot_trajectory_visualization():
    """Generate t-SNE visualization of denoising trajectories showing task-specific clusters."""
    
    try:
        from sklearn.manifold import TSNE
        from sklearn.decomposition import PCA
    except ImportError:
        print("Warning: sklearn not available. Installing...")
        import subprocess
        subprocess.check_call(['pip', 'install', 'scikit-learn'])
        from sklearn.manifold import TSNE
        from sklearn.decomposition import PCA
    
    # Simulate trajectory embeddings for different tasks
    # In practice, these would come from actual model embeddings at different diffusion steps
    np.random.seed(42)
    
    n_samples_per_task = 50
    n_steps_per_trajectory = 10  # Sample 10 steps per trajectory
    
    tasks = ['SST-2', 'SQuAD', 'XSum', 'AGNews']
    colors = ['steelblue', 'coral', 'green', 'purple']
    
    # Generate synthetic trajectory data
    # Each task has distinct clusters in embedding space
    all_embeddings = []
    all_labels = []
    all_tasks = []
    
    for task_idx, (task, color) in enumerate(zip(tasks, colors)):
        # Create task-specific cluster center
        center = np.random.randn(128) * 5  # 128-dim embedding
        center[task_idx * 32:(task_idx + 1) * 32] += 10  # Make task-specific dimensions prominent
        
        for sample_idx in range(n_samples_per_task):
            # Generate trajectory: start from noise, converge to task-specific region
            for step in range(n_steps_per_trajectory):
                # Progressively move from random noise to task center
                noise_level = 1.0 - (step / n_steps_per_trajectory)
                embedding = center + np.random.randn(128) * (2 + 3 * noise_level)
                all_embeddings.append(embedding)
                all_labels.append(step)
                all_tasks.append(task)
    
    all_embeddings = np.array(all_embeddings)
    
    # Apply PCA first for dimensionality reduction (faster)
    print("Applying PCA...")
    pca = PCA(n_components=50)
    embeddings_pca = pca.fit_transform(all_embeddings)
    
    # Apply t-SNE
    print("Applying t-SNE (this may take a minute)...")
    tsne = TSNE(n_components=2, random_state=42, perplexity=30, max_iter=1000)
    embeddings_2d = tsne.fit_transform(embeddings_pca)
    
    # Create figure
    fig, ax = plt.subplots(figsize=(12, 10))
    
    # Plot each task with different colors
    for task, color in zip(tasks, colors):
        task_mask = np.array(all_tasks) == task
        task_embeddings = embeddings_2d[task_mask]
        task_labels = np.array(all_labels)[task_mask]
        
        # Plot points, colored by diffusion step (darker = later step)
        scatter = ax.scatter(task_embeddings[:, 0], task_embeddings[:, 1], 
                            c=task_labels, cmap='viridis', s=30, 
                            alpha=0.6, edgecolors=color, linewidths=0.5,
                            label=task)
    
    # Add colorbar for diffusion steps
    cbar = plt.colorbar(ax.collections[0], ax=ax)
    cbar.set_label('Diffusion Step (darker = later)', fontsize=12)
    
    ax.set_xlabel('t-SNE Dimension 1', fontsize=14, fontweight='bold')
    ax.set_ylabel('t-SNE Dimension 2', fontsize=14, fontweight='bold')
    ax.set_title('t-SNE Visualization of Denoising Trajectories\n(Task-Specific Clusters)', 
                 fontsize=16, fontweight='bold')
    ax.legend(loc='upper right', fontsize=11, framealpha=0.9)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    # Save as PNG and PDF
    output_path_png = figures_dir / "trajectory_visualization.png"
    output_path_pdf = figures_dir / "trajectory_visualization.pdf"
    plt.savefig(output_path_png, dpi=300, bbox_inches='tight')
    plt.savefig(output_path_pdf, bbox_inches='tight')
    print(f"Saved: {output_path_png}")
    print(f"Saved: {output_path_pdf}")
    plt.show()

plot_trajectory_visualization()

## Summary

All required figures have been generated and saved to `doc/figures/`:

1. ✅ **rank_ablation.png/pdf**: Rank vs. performance and trainable parameters
2. ✅ **effective_rank.png/pdf**: Effective rank across diffusion steps
3. ✅ **data_efficiency.png/pdf**: Performance vs. training data size
4. ✅ **trajectory_visualization.png/pdf**: t-SNE visualization of denoising trajectories

**Note**: The current visualizations use example/synthetic data. To use real experimental data:
- Update the data loading functions to read from actual experiment outputs
- Replace synthetic data arrays with real measurements
- Adjust parameters to match your experimental setup

## Conclusion

This notebook provides tools for analyzing LoRA-Diffusion results. Key analyses:

1. **Training curves**: Monitor loss and accuracy over time
2. **Method comparison**: Compare different PEFT methods
3. **Parameter efficiency**: Analyze trainable parameters vs. performance
4. **Step-adaptive ranks**: Visualize rank allocation strategy
5. **Multi-task results**: Compare performance across tasks
6. **Training time**: Analyze computational efficiency

**Next steps:**
- Update paths to your actual experiment directories
- Run your own analyses
- Export results for papers/presentations
- Customize visualizations as needed